In [23]:
from sqlalchemy import create_engine
import pandas as pd
import timeit
import psycopg2 
from pgspecial.main import PGSpecial
from pgspecial.namedqueries import NamedQueries
import io
import datetime

In [9]:
# If you turn this feature on, you can display each 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In-line SQL

In [10]:
%load_ext sql

In [3]:
%%sql postgresql://weaves:3lbpb84@j1/weaves
select count(*) from actor

1 rows affected.


count
200


In [4]:
version = %sql select version()

 * postgresql://weaves:***@j1/weaves
1 rows affected.


In [5]:
version

version
"PostgreSQL 11.5 (Debian 11.5-1+deb10u1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit"


In [13]:
engine = create_engine('postgresql+psycopg2://weaves:3lbpb84@j1:5432/weaves')

In [11]:
tbl0 = 'actor'

In [12]:
df0 = pd.read_sql_query("SELECT * FROM %s;" % tbl0, engine)

In [15]:
df0 = pd.read_sql_table(tbl0, engine)

In [18]:
df0.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [2]:
timeit.timeit('"-".join(str(n) for n in range(100))', number=10000)

0.439078084891662

In [5]:
timeit.timeit('import pandas as pd; pd.read_csv("cache/in/transactions.csv2")', number=1)

13.19480722793378

In [6]:
trns = pd.read_csv("cache/in/transactions.csv2")

In [8]:
trns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2740075 entries, 0 to 2740074
Data columns (total 12 columns):
transaction_id           int64
transactions_type        object
transactions_currency    object
amount_usd               float64
transactions_state       object
ea_cardholderpresence    object
ea_merchant_mcc          float64
ea_merchant_city         float64
ea_merchant_country      object
direction                object
user_id                  int64
created_date             object
dtypes: float64(3), int64(2), object(7)
memory usage: 250.9+ MB


In [44]:
# It is prohibitively slow to use Panda, sqlalchemy and psycopg2 to upload to a database.
# It is possible to have Pandas create the schema and use a database command to upload.
# This creates the schema.
trns.head(1).to_sql('trns', engine, if_exists='replace', chunksize=4000, method='multi', index=False)

In [45]:
%%sql postgresql://weaves:3lbpb84@j1/weaves
select count(*) from trns

1 rows affected.


count
1


In [46]:
# Check the schema. Postgres doesn't support DESCRIBE.
# There is a special command \d trns, but pgspecial does not work with the %sql magic.
%%sql
SELECT
   *
FROM
   information_schema.COLUMNS
WHERE
   TABLE_NAME = 'trns';

 * postgresql://weaves:***@j1/weaves
12 rows affected.


table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,numeric_precision,numeric_precision_radix,numeric_scale,datetime_precision,interval_type,interval_precision,character_set_catalog,character_set_schema,character_set_name,collation_catalog,collation_schema,collation_name,domain_catalog,domain_schema,domain_name,udt_catalog,udt_schema,udt_name,scope_catalog,scope_schema,scope_name,maximum_cardinality,dtd_identifier,is_self_referencing,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
weaves,public,trns,transaction_id,1,None,YES,bigint,None,None,64,2,0,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,int8,None,None,None,None,1,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,transactions_type,2,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,2,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,transactions_currency,3,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,3,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,amount_usd,4,None,YES,double precision,None,None,53,2,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,float8,None,None,None,None,4,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,transactions_state,5,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,5,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,ea_cardholderpresence,6,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,6,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,ea_merchant_mcc,7,None,YES,double precision,None,None,53,2,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,float8,None,None,None,None,7,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,ea_merchant_city,8,None,YES,double precision,None,None,53,2,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,float8,None,None,None,None,8,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,ea_merchant_country,9,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,9,NO,NO,None,None,None,None,None,NO,NEVER,None,YES
weaves,public,trns,direction,10,None,YES,text,None,1073741824,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,weaves,pg_catalog,text,None,None,None,None,10,NO,NO,None,None,None,None,None,NO,NEVER,None,YES


In [41]:
%%sql
truncate table trns;
select count(*) from trns

 * postgresql://weaves:***@j1/weaves
Done.
1 rows affected.


count
0


In [49]:
# This is a client-side pgspecial command, it only works from the command-line
%sql \copy trns FROM 't.csv' WITH DELIMITER ',' CSV HEADER

 * postgresql://weaves:***@j1/weaves


AttributeError: 'NoneType' object has no attribute 'fetchall'